In [1]:
import pickle
import numpy as np
from rich.progress import track

In [2]:
wifi_train = np.load("../data/working/train_wifi_features.npy")
wifi_test = np.load("../data/working/test_wifi_features.npy")

In [7]:
bssid_uniques = np.unique(np.concatenate([wifi_train[:, 0], wifi_test[:, 0]], axis=0).ravel())

In [8]:
with open("../data/working/bssid_map.pkl", "rb") as file:
    bssid_map = pickle.load(file)

In [9]:
encoded_bssid = [
        [bssid_map[d] for d in row[0]]
        for row in track(wifi_train, total=len(wifi_train))
    ]

Output()

In [10]:
bssid_map = {bssid: int(i + 1) for i, bssid in enumerate(bssid_uniques)}

In [12]:
%%time

from rich.progress import track

encoded_bssid = [[bssid_map[d] for d in row[0]] for row in track(wifi_train, total=len(wifi_train))]

Output()

CPU times: user 7.68 s, sys: 194 ms, total: 7.87 s
Wall time: 7.83 s


In [15]:
wifi_train[:, 0] = np.array(encoded_bssid)

In [18]:
wifi_train = wifi_train.astype('int64')

In [19]:
wifi_train

array([[[153239,  68022,  55328, ..., 185859, 185859, 185859],
        [   -52,    -53,    -53, ...,   -999,   -999,   -999],
        [  5785,   2437,   5785, ...,      0,      0,      0]],

       [[ 60991, 177997, 177997, ...,   4660,   1396,  62079],
        [   -42,    -48,    -48, ...,    -71,    -71,    -71],
        [  2462,   2462,   2462, ...,   2437,   5785,   5785]],

       [[ 60991,  60991, 177997, ...,   4660,  62079, 102691],
        [   -42,    -47,    -48, ...,    -69,    -69,    -70],
        [  2462,   2462,   2462, ...,   2437,   5785,   2437]],

       ...,

       [[108257, 152070, 152070, ..., 151613, 162246, 162667],
        [   -33,    -38,    -38, ...,    -75,    -75,    -76],
        [  2427,   5180,   5180, ...,   5765,   2462,   2472]],

       [[108257, 152070, 152070, ..., 151613, 162246, 162667],
        [   -33,    -38,    -38, ...,    -75,    -75,    -76],
        [  2427,   5180,   5180, ...,   5765,   2462,   2472]],

       [[108257, 152070, 152070,

In [21]:
def encode_bssid(wifi_feature: np.ndarray) -> np.ndarray:
    with open("../data/working/bssid_map.pkl", "rb") as file:
        bssid_map = pickle.load(file)
    encoded_bssid = [
        [bssid_map[d] for d in row[0]]
        for row in track(wifi_feature, total=len(wifi_feature))
    ]
    wifi_feature[:, 0] = np.array(encoded_bssid)
    return wifi_feature

In [22]:
wifi_train = np.load("../data/working/train_wifi_features.npy")

In [23]:
wifi_train = encode_bssid(wifi_train)

Output()

In [27]:
wifi_train = wifi_train.astype("int64")

In [30]:
wifi_train.dtype

dtype('int64')

In [ ]:



def wifi_preprocessing():
    # Train data
    filepath = "../data/working/train_wifi_features.npy"
    if not pathlib.Path(filepath).exists():
        with timer("Dump wifi of train"):
            wifi_features = get_wifi_from_waypoints_in_parallel()
            np.save(filepath, wifi_features)

    # Test data
    filepath = "../data/working/test_wifi_features.npy"
    if not pathlib.Path(filepath).exists():
        with timer("Dump wifi of test"):
            wifi_features = get_test_wifi_from_waypoints_in_parallel()
            np.save(filepath, wifi_features)

    # Label encoding
    dump_wifi_bssid_map()
    filepath = "../data/working/train_encoded_wifi_feature.npy"
    if not pathlib.Path(filepath).exists():
        wifi_features = np.load("../data/working/train_wifi_features.npy")
        wifi_features = encode_bssid(wifi_features).astype("int64")
        np.save(filepath, wifi_features)

    filepath = "../data/working/test_wifi_encoded_feature.npy"
    if not pathlib.Path(filepath).exists():
        wifi_features = np.load("../data/working/test_wifi_features.npy")
        wifi_features = encode_bssid(wifi_features)
        wifi_features = wifi_features.astype("int64")
        np.save(filepath, wifi_features)